# JEM092 Asset Pricing - Homework 2
### Summer Semester 2021/2022

### Authors: *Petr Čala, Tereza Čechová, Vilém Krejcar*
___

First things first, we install and load the necessary packages. Then we arbitrarily set a seed for reproducibility.

In [126]:
# Required packages
packages <- c("stringr", "quantmod", "tseries", "dplyr", "xml2", "httr", "PortfolioAnalytics")

# Install packages not yet installed
installed_packages <- packages %in% rownames(installed.packages())
if (any(installed_packages == FALSE)) {
  install.packages(packages[!installed_packages])
  print(paste("Installing package ", packages[!installed_packages],"...", sep = ""))
}

# Packages loading
invisible(lapply(packages, library, character.only = TRUE))
rm(list = ls()) #Clean environment

# Miscellaneous
options(repr.plot.width = 6, repr.plot.height = 5)
set.seed(420)

### Groundwork

Although we were able to obtain the data in the first homework, **we choose to use the provided dataset**, as this allows for easier and **more reliable reproducibility**. First we load the data, subset the necessary stocks, save these as new variables, and then discard the source data in order to alleviate some memory.

In [154]:
# Static variables
group_number <- 61505008
ticker_file <- "data/61505008_data_download.csv"
data_file <- "data/Asset_Pricing_HW_2_data.RData"
group_factor <- "data/third_factor_rand.csv"
data_files <- list(ticker_file, data_file, group_factor)

In [155]:
 # Set wd to project root
if (str_detect(getwd(), 'scripts')) {
    setwd('..')
}

# Assert presence of all required files
for (file in data_files) {
    if (!file.exists(file)) {
        print(paste0('File ', file, ' does not exist or has been misplaced.'))
    }
}

# Load tickers of required stocks
tickers <- as.character(unlist(read.csv(ticker_file)[2]))

# Check the assigned group factor
getFactor <- function(factor){
    data <- read.csv(factor)
    matching_row <- which(data[2] == group_number)
    assigned_factor <- as.character(data[matching_row, 3])
    print(paste0('The factor assigned to this group is ', assigned_factor, '.'))
}
getFactor(group_factor)

[1] "The factor assigned to this group is VOL."


In [130]:
# Load source data into the working directory
source_data <- load(data_file)

# Subset only for the stocks we will need
# Note - use these as BV$AAPL, not BV['AAPL']
BV <- book_value_sap500[tickers]
MKT <- MktCap_sap500[tickers]
OHLCV <- OHLCV_sap500[tickers]

# Remove source data from working directory for better memory management
rm(list = source_data)

### Portfolio analysis

In [135]:
head(BV$SJM)

           Stock Price Book Value per Share Price/Book Ratio
2006-01-31       25.97                29.93             0.87
2006-04-30       23.59                30.34             0.78
2006-07-31       27.00                30.49             0.89
2006-10-31       29.83                30.90             0.97
2007-01-31       29.08                31.04             0.94
2007-04-30       34.38                31.62             1.09